学习 Spark 的 python api

In [4]:
from pyspark import SparkContext
import os
os.environ["SPARK_HOME"] = "/home/hadoop/spark-2.0.1-bin-hadoop2.7"   #KeyError: 'SPARK_HOME'
sc=SparkContext(appName='Test')


#conf = (SparkConf().setMaster('local').setAppName('a'))

